<a href="https://colab.research.google.com/github/platypus2000jp/Colaboratory/blob/main/%E8%96%AC%E8%8D%89%E3%81%AE%E6%A0%BD%E5%9F%B9%E8%A9%A6%E9%A8%93%E3%81%AE%E5%AE%9F%E9%A8%93%E8%A8%88%E7%94%BB%E3%80%80%E6%95%B0%E7%90%86%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AB%E3%81%8A%E3%81%91%E3%82%8B%E6%9C%80%E5%B0%8F%E5%8C%96%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: 薬草の栽培試験をしたいので最適化計画で最小の試験数になるように試験をデザインして、試験数と組み合わせを生成してください。データは適当なものを栽培しますが、種名は実在のハーブとします。実験計画はPuLPを利用してください。試験数は何回ですか。

!pip install pulp

from pulp import *
import itertools

# 薬草の種類
herbs = ["ラベンダー", "カモミール", "ローズマリー", "セージ", "オレガノ"]

# 土壌の種類
soils = ["砂質土壌", "粘土質土壌", "腐葉土"]

# 水やり量
waters = ["少量", "適量", "多量"]

# 組み合わせをすべて列挙
combinations = list(itertools.product(herbs, soils, waters))

# 最適化問題を定義
problem = LpProblem("HerbalCultivationExperiment", LpMinimize)

# 決定変数 (各組み合わせの試験を行うかどうか)
variables = LpVariable.dicts("Experiment", combinations, cat=LpBinary)

# 目的関数: 試験数を最小化
problem += lpSum([variables[combination] for combination in combinations])

# 制約条件: 各薬草、土壌、水やり量について、少なくとも1回は試験を行う
for herb in herbs:
  problem += lpSum([variables[(herb, soil, water)] for soil in soils for water in waters]) >= 1

for soil in soils:
  problem += lpSum([variables[(herb, soil, water)] for herb in herbs for water in waters]) >= 1

for water in waters:
  problem += lpSum([variables[(herb, soil, water)] for herb in herbs for soil in soils]) >= 1


# 問題を解く
status = problem.solve()

# 結果を表示
print("Status:", LpStatus[status])

if status == LpStatusOptimal:
  print("試験数:", value(problem.objective))
  for combination in combinations:
    if value(variables[combination]) == 1:
      print("組み合わせ:", combination)



Status: Optimal
試験数: 5.0
組み合わせ: ('ラベンダー', '腐葉土', '少量')
組み合わせ: ('カモミール', '腐葉土', '少量')
組み合わせ: ('ローズマリー', '腐葉土', '多量')
組み合わせ: ('セージ', '粘土質土壌', '適量')
組み合わせ: ('オレガノ', '砂質土壌', '少量')
